# Is Julia fast?

Julia isn't fast *per se*.

One can write terribly slow code in any language, including Julia.

So let's ask a different question.

# *Can* Julia be fast?

#### A quick speed comparison

Note that the clean and concise Julia implementation is **beating numpy's C implementation for small matrices** and is **on-par for large matrix sizes**.

At the same time, the Julia code is *generic* and works for arbitrary types!

It even works for non-numerical types. The only requirement is that the type has a *one* (identity element) and a multiplication operation defined.

Here, `one(String) == ""` since the empty string is the identity under multiplication (string concatenation).

# How can Julia be fast?

 
**AST = Abstract Syntax Tree**

**SSA = Static Single Assignment**

**[LLVM](https://de.wikipedia.org/wiki/LLVM) = Low Level Virtual Machine**

### Specialization and code inspection

**Julia specializes on the types of function arguments.**

When a function is called for the first time, Julia compiles efficient machine code for the given input types.

If it is called again, the already existing machine code is reused, until we call the function with different input types.

In [1]:
func(x,y) = x^2 + y

func (generic function with 1 method)

In [ ]:
@code

In [3]:
@time func(1,2)
@time func(1,2)

  0.000003 seconds (4 allocations: 160 bytes)
  0.000003 seconds (4 allocations: 160 bytes)


3

**First call:** compilation + running the code

**Second call:** running the code

In [ ]:
@time func(1,2)

If the input types change, Julia compiles a new specialization of the function.

In [6]:
@time func(1.3,4.8)
@time func(1.3,4.8)

  0.000002 seconds (5 allocations: 176 bytes)
  0.000004 seconds (5 allocations: 176 bytes)


6.49

We now have two efficient codes, one for all `Int64` inputs and another one for all `Float64` arguments, in the cache.

### *But I really want to see what happens!*

We can inspect the code at all transformation stages with a bunch of macros:

* The AST after parsing (**`@macroexpand`**)
* The AST after lowering (**`@code_typed`**, **`@code_warntype`**)
* The AST after type inference and optimization (**`@code_lowered`**)
* The LLVM IR (**`@code_llvm`**)
* The assembly machine code (**`@code_native`**)

In [7]:
@code_typed func(1,2)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.add_int(%1, y)::Int64
└──      return %2
) => Int64

In [8]:
@code_lowered func(1,2)

CodeInfo(
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(Main.:^, x, %2)
│   %4 = %3 + y
└──      return %4
)

In [9]:
@code_llvm func(1,2)


;  @ In[1]:1 within `func'
define i64 @julia_func_16454(i64, i64) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %2 = mul i64 %0, %0
; └└
; ┌ @ int.jl:53 within `+'
   %3 = add i64 %2, %1
; └
  ret i64 %3
}


We can remove the comments (lines starting with `;` using `debuginfo=:none`).

In [10]:
@code_llvm debuginfo=:none func(1,2)


define i64 @julia_func_16454(i64, i64) {
top:
  %2 = mul i64 %0, %0
  %3 = add i64 %2, %1
  ret i64 %3
}


In [11]:
@code_native func(1,2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[1]:1 within `func'
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ In[1]:1 within `*'
	imulq	%rdi, %rdi
; │└└
; │┌ @ int.jl:53 within `+'
	leaq	(%rdi,%rsi), %rax
; │└
	retq
	nopl	(%rax)
; └


Let's compare this to `Float64` input.

In [ ]:
@code_native func(1.2,2.9)

## How important is code specialization?

Let's try to estimate the performance gain by specialization.

We wrap our numbers into a custom type which internally stores them as `Any` to prevent specialization.

(This is qualitatively comparable to what Python does.)

In [12]:
struct Anything
    value::Any
end

struct Specialized{T<:Number}
   value::T 
end

operation(x::Specialized) = x.value^2 + sqrt(x.value)
operation(x::Anything) = x.value^2 + sqrt(x.value)

operation (generic function with 2 methods)

In [14]:
using BenchmarkTools

y = Specialized(2.0)
@btime operation($y);

x = Anything(2.0)
@btime operation($x);

  1.853 ns (0 allocations: 0 bytes)
  67.373 ns (3 allocations: 48 bytes)


**That's about an 40 times slowdown!**

In [15]:
@code_native operation(y)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[12]:9 within `operation'
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ In[12]:9 within `*'
	pushq	%rax
	vmovsd	(%rdi), %xmm0           ## xmm0 = mem[0],zero
; │└└
; │┌ @ math.jl:493 within `sqrt'
; ││┌ @ float.jl:452 within `<'
	vxorps	%xmm1, %xmm1, %xmm1
	vucomisd	%xmm0, %xmm1
; ││└
	ja	L29
; │└
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ float.jl:399 within `*'
	vmulsd	%xmm0, %xmm0, %xmm1
; │└└
; │┌ @ math.jl:494 within `sqrt'
	vsqrtsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ float.jl:395 within `+'
	vaddsd	%xmm0, %xmm1, %xmm0
; │└
	popq	%rax
	retq
; │┌ @ math.jl:493 within `sqrt'
L29:
	movabsq	$throw_complex_domainerror, %rax
	movabsq	$4446453176, %rdi       ## imm = 0x109077DB8
	callq	*%rax
	ud2
	nopw	%cs:(%rax,%rax)
; └└


In [16]:
@code_native operation(x)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[12]:10 within `operation'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$96, %rsp
	movq	%rsi, %rbx
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 32(%rsp)
	movq	%rbx, 72(%rsp)
	movabsq	$jl_get_ptls_states_fast, %rax
	vzeroupper
	callq	*%rax
	movq	%rax, %r14
	movq	$4, 32(%rsp)
	movq	(%r14), %rax
	movq	%rax, 40(%rsp)
	leaq	32(%rsp), %rax
	movq	%rax, (%r14)
	movq	(%rbx), %r15
; │┌ @ Base.jl:20 within `getproperty'
	movq	(%r15), %rax
; │└
	movq	-8(%rax), %rcx
	movabsq	$jl_system_image_data, %rdx
	movq	%rdx, (%rsp)
	movabsq	$jl_system_image_data, %rdx
	movq	%rdx, 8(%rsp)
	shrq	$4, %rcx
	cmpq	$281079417, %rcx        ## imm = 0x10C0EE79
	je	L177
	movq	%rax, 16(%rsp)
	movabsq	$jl_system_image_data, %rax
	movq	%rax, 24(%rsp)
	movabsq	$jl_apply_generic, %rax
	movq	%rsp, %rdi
	movl	$4, %esi
	callq	*%rax
	movq	%rax, %rbx
	jmp	L240
L177:
	movabsq	$jl_system_image_data, %rax
	movq	%rax, 16(%

# Make run-time the fun time.

In scientific computations, we typically run a piece of code many times over and over again. Think of a Monte Carlo simulation, for example, where we perform the update and the Metropolis check millions of times.

**Therefore, we want our run-time to be as short as possible.**

On the other hand, for a given set of input arguments, Julia compiles the piece of code only once, as we have seen above. The time it takes to compile our code is almost always negligible compared to the duration of the full computation.

A general strategy is therefore to move parts of the computation to compile-time.

Since Julia specializes on types, at compile-time **only type information is available to the compiler.**

In [ ]:
f1(x::Int) = x + 1
f2(x::Int) = x + 2

function f_slow(x::Int, p::Bool)
    if p                                # check depends on the value of p
        return f1(x)
    else
        return f2(x)
    end
end

In [ ]:
@code_llvm debuginfo=:none f_slow(1, true)

We can eliminate the if branch by moving the condition check to the type domain. This way, it **will only be evaluated once at compile-time.**

In [ ]:
abstract type Boolean end
struct True <: Boolean end # type domain true
struct False <: Boolean end # type domain false

function f_fast(x::Int, p::Boolean)
    if typeof(p) == True                # check solely based on the type of p
        return f1(x)
    else
        return f2(x)
    end
end

In [ ]:
@code_llvm debuginfo=:none f_fast(1, True())

# Are explicit type annotations necessary? (like in C or Fortran)

Note that Julia's type inference is powerful. Specifying types **is not** necessary for best performance!

In [ ]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

In [ ]:
@btime my_function(10);
@btime my_function_typed(10);

 However, annotating types explicitly can serve a purpose.

* **Define a user interface/type filter** (will throw error if incompatible type is given)
* Enforce conversions
* Rarely, help the compiler infer types in tricky situations

# Core messages of this Notebook

* Julia **can be fast.**
* **A function is compiled when called for the first time** with a given set of argument types.
* The are **multiple compilation steps** all of which can be inspected through macros like `@code_warntype`.
* **Code specialization** based on the types of all of the input arguments is important for speed.
* Calculations can be moved to compile-time to make run-time faster.
* In virtually all cases, **explicit type annotations are irrelevant for performance**.
* Type annotations in function signatures define a **type filter/user interface**.